In [1]:
import os
import torch
import torchtext
from torchtext.datasets import translation, imdb, language_modeling, nli
from torchtext.datasets import sequence_tagging, unsupervised_learning, text_classification, sst
import pytorch_lightning as pl

import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import random_split, DataLoader, Dataset

In [2]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
DATA_ROOT_PATH = '../data/text_classification/'
NGRAMS = 2
VOCAB = None

BATCH_SIZE = 16
EMBED_DIM = 32

In [5]:
train_dataset, test_dataset = text_classification.SogouNews(root=DATA_ROOT_PATH, ngrams=NGRAMS, vocab=VOCAB)
VOCAB_SIZE = len(train_dataset.get_vocab())
NUN_CLASS = len(train_dataset.get_labels())

450000lines [03:47, 1979.92lines/s]
450000lines [07:31, 995.91lines/s] 
60000lines [01:01, 970.66lines/s] 


In [8]:
train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = random_split(train_dataset, [train_len, len(train_dataset) - train_len])

train_loader = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch, num_workers=8)
valid_loader = DataLoader(sub_valid_, batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch, num_workers=8)



In [40]:
lengths, text = train_dataset[10]

text.shape

torch.Size([303])

In [43]:
texts, offsets, cls = next(iter(train_loader))


In [44]:
vocab = train_dataset.get_vocab()
print([vocab.itos[token] for token in texts.tolist()])

['yi1', 'ji4', 'du4', 'ho2ng', 'gua1n', 'ji1ng', 'ji4', 'yu4n', 'ha2ng', 'fa1', 'bu4', 'hui4', 'dia3n', 'pi2ng', 'zhi1', 'ti2', 'ga1o', 'sha4ng', 'wa3ng', 'dia4n', 'jia4', ',', 'xia1o', 'sho4u', 'dia4n', 'jia4', 'bu4', 'do4ng', 'bi3', 'jia4o', 'ke3', 'xi2ng', ',', 'da4n', 'shi4', 'xia4o', 'guo3', 'yo3u', 'xia4n', '.', 'wo3', 'me2n', 're4n', 'we2i', ',', 'shi4', 'da4ng', 'ti2', 'ga1o', 'sha4ng', 'wa3ng', 'dia4n', 'jia4', ',', 'xia1o', 'sho4u', 'dia4n', 'jia4', 'bu4', 'do4ng', ',', 'to2ng', 'shi2', 'dui4', 'dia4n', 'wa3ng', 'qi3', 'ye4', 'ji4n', 'ha2ng', 'bu3', 'tie1', ',', 'shi4', 'yi2', 'ge4', 'jia4o', 'ha3o', 'di2', 'jie3', 'jue2', 'fa1', 'dia4n', 'qi3', 'ye4', 'ku4n', 'ji4ng', 'di2', 'fa1ng', 'shi4', '.', 'da4n', 'qia2n', 'ti2', 'shi4', 'bu4', 'ne2ng', 'ra4ng', 'dia4n', 'wa3ng', 'qi3', 'ye4', 'su3n', 'shi1', 'ta4i', 'da4', '.', '2007', 'nia2n', 'dia4n', 'wa3ng', 'qi3', 'ye4', 'li4', 'ru4n', 'zo3ng', 'e2', '650', 'yi4', 'zuo3', 'yo4u', ',', 'qua2n', 'guo2', 'fa1', 'dia4n', 'lia2ng', '

In [33]:
texts.shape, offsets.shape, cls.shape


(torch.Size([10144]), torch.Size([16]), torch.Size([16]))

In [7]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [27]:
from pytorch_lightning.metrics import Accuracy

class TaskTextSentiment(pl.LightningModule):
    def __init__(self, model, optimizers, criterion, scheduler=None):
        super().__init__()
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion
        self.scheduler = scheduler
        self.metric = Accuracy()
        
    
    def shared_step(self, batch, batch_idx):
        text, offsets, cls = batch
        output = self.model(text, offsets)
        loss = self.criterion(output, cls)
#         acc = (output.argmax(1) == cls).sum().item()
        acc = self.metric(output.argmax(1), cls)
        return loss, acc

    
    def training_step(self, batch, batch_idx):
        loss, acc = self.shared_step(batch, batch_idx)
        result = pl.TrainResult(loss)
#         result.log_dict({'trn_loss': loss})
        result.log_dict({'trn_loss': loss, 'trn_acc':acc})
        
        return result
    
    def validation_step(self, batch, batch_idx):
        loss, acc = self.shared_step(batch, batch_idx)
        result = pl.EvalResult(checkpoint_on=loss)
#         result.log_dict({'val_loss': loss})
        result.log_dict({'val_loss': loss, 'val_acc': acc})
        
        return result
    
    def configure_optimizers(self):
        if self.scheduler:
            return [self.optimizer], [self.scheduler]
        return self.optimizer
    
  

In [28]:
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)
# model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [30]:
from pytorch_lightning import loggers as pl_loggers

tb_logger = pl_loggers.TensorBoardLogger('logs/sogoue')
task = TaskTextSentiment(model, optimizer, criterion, scheduler)
trainer = pl.Trainer(gpus=1, logger=tb_logger)
trainer.fit(task, train_loader, valid_loader)

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | TextSentiment    | 99 M  
1 | criterion | CrossEntropyLoss | 0     
2 | metric    | Accuracy         | 0     
INFO:lightning:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | TextSentiment    | 99 M  
1 | criterion | CrossEntropyLoss | 0     
2 | metric    | Accuracy         | 0     


Saving latest checkpoint..
INFO:lightning:Saving latest checkpoint..


1

In [10]:
from torch.utils.data import DataLoader
def train_func(sub_train_):
    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [11]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 49 seconds
	Loss: 0.0128(train)	|	Acc: 93.8%(train)
	Loss: 0.0000(valid)	|	Acc: 95.8%(valid)
Epoch: 2  | time in 0 minutes, 52 seconds
	Loss: 0.0079(train)	|	Acc: 96.1%(train)
	Loss: 0.0000(valid)	|	Acc: 96.2%(valid)
Epoch: 3  | time in 0 minutes, 52 seconds
	Loss: 0.0064(train)	|	Acc: 96.8%(train)
	Loss: 0.0000(valid)	|	Acc: 96.1%(valid)
Epoch: 4  | time in 0 minutes, 53 seconds
	Loss: 0.0055(train)	|	Acc: 97.3%(train)
	Loss: 0.0000(valid)	|	Acc: 96.4%(valid)
Epoch: 5  | time in 0 minutes, 53 seconds
	Loss: 0.0048(train)	|	Acc: 97.6%(train)
	Loss: 0.0000(valid)	|	Acc: 95.6%(valid)
